<a href="https://colab.research.google.com/github/Vanshkumar16/ml_model/blob/main/Crop_grading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("kamal01/top-agriculture-crop-disease")

print("Path to dataset files:", path)

100%|██████████| 4.33G/4.33G [01:23<00:00, 55.4MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/kamal01/top-agriculture-crop-disease/versions/1


In [3]:
import os
dataset_contents = os.listdir('/root/.cache/kagglehub/datasets/kamal01/top-agriculture-crop-disease/versions/1')
print("Contents of /root/.cache/kagglehub/datasets/kamal01/top-agriculture-crop-disease/versions/1")
for item in dataset_contents:
    print(item)

Contents of /root/.cache/kagglehub/datasets/kamal01/top-agriculture-crop-disease/versions/1
Crop Diseases


In [5]:
import os
try:
    downloaded_root_contents = os.listdir('/root/.cache/kagglehub/datasets/kamal01/top-agriculture-crop-disease/versions/1')
    print("Contents of /root/.cache/kagglehub/datasets/kamal01/top-agriculture-crop-disease/versions/1")
    for item in downloaded_root_contents:
      print(item)
except FileNotFoundError:
    print("Error: Directory not found at '/root/.cache/kagglehub/datasets/kamal01/top-agriculture-crop-disease/versions/1'")
except Exception as e:
    print(f"An error occurred while listing directory contents: {e}")

Contents of /root/.cache/kagglehub/datasets/kamal01/top-agriculture-crop-disease/versions/1
Crop Diseases


In [9]:
import os

dataset_root_path = '/root/.cache/kagglehub/datasets/kamal01/top-agriculture-crop-disease/versions/1'
dataset_subdir = os.path.join(dataset_root_path, 'Dataset')

print(f"Contents of {dataset_subdir}:")
try:
    for item in os.listdir(dataset_subdir):
        print(item)
except FileNotFoundError:
    print(f"Error: Directory not found at '{dataset_subdir}'")
except Exception as e:
    print(f"An error occurred while listing directory contents: {e}")

Contents of /root/.cache/kagglehub/datasets/kamal01/top-agriculture-crop-disease/versions/1/Dataset:
Error: Directory not found at '/root/.cache/kagglehub/datasets/kamal01/top-agriculture-crop-disease/versions/1/Dataset'


In [12]:
import os

print(f"Contents of {dataset_root_path}:")
try:
    for item in os.listdir(dataset_root_path):
        print(item)
except FileNotFoundError:
    print(f"Error: Directory not found at '{dataset_root_path}'")
except Exception as e:
    print(f"An error occurred while listing directory contents: {e}")

Contents of /root/.cache/kagglehub/datasets/kamal01/top-agriculture-crop-disease/versions/1:
Crop Diseases


In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from PIL import Image
import numpy as np
import os
import shutil
import random


In [15]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [16]:
IMG_HEIGHT = 128
IMG_WIDTH = 128
BATCH_SIZE = 32
NUM_CLASSES = 38
EPOCHS = 10
DATASET_ROOT = '/root/.cache/kagglehub/datasets/kamal01/top-agriculture-crop-disease/versions/1/Crop Diseases/'

In [17]:
NORM_MEAN = [0.485, 0.456, 0.406]
NORM_STD = [0.229, 0.224, 0.225]

In [18]:
# Transforms for training (includes augmentation)
train_transforms = transforms.Compose([
    transforms.Resize((IMG_HEIGHT, IMG_WIDTH)), # Resize all images
    transforms.RandomHorizontalFlip(),         # Data Augmentation
    transforms.RandomRotation(15),             # Data Augmentation
    transforms.ColorJitter(brightness=0.1, contrast=0.1), # Data Augmentation
    transforms.ToTensor(),                     # Convert PIL image to Tensor (C, H, W) and normalize to [0, 1]
    transforms.Normalize(NORM_MEAN, NORM_STD)  # Normalize to [-1, 1] range
])

# Transforms for testing and prediction (no augmentation)
test_transforms = transforms.Compose([
    transforms.Resize((IMG_HEIGHT, IMG_WIDTH)), # Resize all images
    transforms.ToTensor(),                     # Convert PIL image to Tensor
    transforms.Normalize(NORM_MEAN, NORM_STD)
])

In [19]:
#Dataloading and DataLoader
from torch.utils.data import random_split # Import random_split

try:
    print("\nAttempting to load data using PyTorch ImageFolder...")
    full_dataset = datasets.ImageFolder(
        root=DATASET_ROOT,
        transform=train_transforms # Use train_transforms for the full dataset before splitting
    )
    train_size = int(0.8 * len(full_dataset))
    test_size = len(full_dataset) - train_size

    train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])
    inferred_class_names = [k.replace('_', ' ').title() for k in full_dataset.classes]

    # Create dataloaders
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

    print(f"Successfully found {len(inferred_class_names)} classes.")
    print(f"Inferred Class Names (Sample): {inferred_class_names[:5]}...")
    print(f"Training set size: {len(train_dataset)}")
    print(f"Test set size: {len(test_dataset)}")

except Exception as e:
    print(f"\n[ERROR] Data Loading Failed: {e}")
    print(f"Please ensure the dataset is downloaded and extracted into a folder named '{DATASET_ROOT}' with its class subdirectories.")
    inferred_class_names = [f"Class {i+1}" for i in range(NUM_CLASSES)]


Attempting to load data using PyTorch ImageFolder...
Successfully found 17 classes.
Inferred Class Names (Sample): ['Corn   Common Rust', 'Corn   Gray Leaf Spot', 'Corn   Healthy', 'Corn   Northern Leaf Blight', 'Potato   Early Blight']...
Training set size: 10659
Test set size: 2665


In [20]:
#CNN Model Architectures
class CropClassifier(nn.Module):
    def __init__(self, num_classes):
        super(CropClassifier, self).__init__()
        # Input (RGB), 128x128
        self.conv_layer = nn.Sequential(
            # Layer 1: 3 -> 32 filters
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2), #  64x64

            # Layer 2: 32 -> 64 filters
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2), #  32x32

            # Layer 3: 64 -> 64 filters
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)  #  16x16
        )
        self.fc_layer = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(64 * 16 * 16, 256),
            nn.ReLU(),
            nn.Linear(256, num_classes)
        )
    def forward(self, x):
        # x shape: (Batch_Size, 3, 128, 128)
        x = self.conv_layer(x)
        # x shape: (Batch_Size, 64, 16, 16)
        x = x.reshape(x.size(0), -1) # Flatten for linear layers
        # x shape: (Batch_Size, 16384)
        x = self.fc_layer(x)
        # x shape: (Batch_Size, 38) -> raw logits
        return x






In [21]:
model = CropClassifier(NUM_CLASSES).to(DEVICE)
print("\n--- Model Architecture ---")
print(model)



--- Model Architecture ---
CropClassifier(
  (conv_layer): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc_layer): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=16384, out_features=256, bias=True)
    (2): ReLU()
    (3): Linear(in_features=256, out_features=38, bias=True)
  )
)


In [22]:
# Training model
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def train_model(model, train_loader, criterion, optimizer, epochs=EPOCHS):
    print("\n--- Model Training")

    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad() #gradients
            outputs = model(inputs) #Forward
            loss = criterion(outputs, labels); #Loss
            loss.backward() #Backward
            optimizer.step() #Optimizer

            running_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}")

def evaluate_model(model, test_loader):
    print("\n--- Model Evaluation")
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)

            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)#Max output
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Accuracy on test images: {accuracy:.2f}%")
    return accuracy



In [24]:
train_model(model, locals().get('train_loader'), criterion, optimizer)
evaluate_model(model, locals().get('test_loader'))


--- Model Training
Epoch 1/10, Loss: 0.4482
Epoch 2/10, Loss: 0.3973
Epoch 3/10, Loss: 0.3523
Epoch 4/10, Loss: 0.3318
Epoch 5/10, Loss: 0.3106
Epoch 6/10, Loss: 0.3119
Epoch 7/10, Loss: 0.2852
Epoch 8/10, Loss: 0.2926
Epoch 9/10, Loss: 0.2727
Epoch 10/10, Loss: 0.2554

--- Model Evaluation
Accuracy on test images: 88.59%


88.59287054409006

In [25]:
# Prediction Function

def predict_image(image_path: str) -> str:
    model.eval()

    try:
        img = Image.open(image_path).convert('RGB')
        img_tensor = test_transforms(img)
        img_tensor = img_tensor.unsqueeze(0)
        img_tensor = img_tensor.to(DEVICE)
        with torch.no_grad():
            outputs = model(img_tensor)
        probabilities = torch.softmax(outputs, dim=1);
        confidence, predicted_index_tensor = torch.max(probabilities, 1)
        predicted_index = predicted_index_tensor.item()
        predicted_class = inferred_class_names[predicted_index]
        confidence_percent = confidence.item() * 100

        return f"Predicted Condition: {predicted_class} ({confidence_percent:.2f}% Confidence)"

    except FileNotFoundError:
        return f"ERROR: Image file not found at path: {image_path}"
    except Exception as e:
        return f"An error occurred during prediction: {e}"

In [26]:
#  Demonstration of Prediction
print("\n--- Prediction Demonstration")

try:

    if not full_dataset.classes:
        raise ValueError("No class names available to pick a sample image. Please ensure full_dataset is loaded correctly.")

    random_class_name_raw = random.choice(full_dataset.classes)
    sample_class_dir = os.path.join(DATASET_ROOT, random_class_name_raw)

    sample_image_path = None

    image_files = [
        os.path.join(sample_class_dir, item)
        for item in os.listdir(sample_class_dir)
        if os.path.isfile(os.path.join(sample_class_dir, item)) and item.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp'))
    ]

    if image_files:
        sample_image_path = random.choice(image_files)
        print(f"Attempting prediction for sample image: {sample_image_path}")
        predicted_result = predict_image(sample_image_path)
    else:
        predicted_result = f"ERROR: No image files found in the directory '{sample_class_dir}' for demonstration."

except Exception as e:
    if inferred_class_names:
        simulated_index = random.randint(0, len(inferred_class_names) - 1)
        simulated_grade = inferred_class_names[simulated_index]
        predicted_result = f"Predicted Condition: {simulated_grade} (Simulated Result: 95.00% Confidence) - Error during actual prediction: {e}"
    else:
        predicted_result = f"An error occurred and no class names were inferred: {e}"

print(f"\nResult for a new phone photo: {predicted_result}")


--- Prediction Demonstration
Attempting prediction for sample image: /root/.cache/kagglehub/datasets/kamal01/top-agriculture-crop-disease/versions/1/Crop Diseases/Rice___Healthy/IMG_20190424_131533.jpg

Result for a new phone photo: Predicted Condition: Rice   Healthy (77.54% Confidence)


In [27]:
import pickle
with open('model12.pkl', 'wb') as file:
    pickle.dump(model, file)

print("Model saved as 'model12.pkl'")

Model saved as 'model12.pkl'
